In [1]:
import pandas as pd
df = pd.read_csv("c:/data/customer/customer.csv")
df.columns

Index(['Int'l Plan', 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls',
       'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins',
       'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn'],
      dtype='object')

In [2]:
cols=["Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins',
    'Day Calls','Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge',
    'Night Mins','Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 
    'Intl Charge','CustServ Calls']
X=df[cols]
y=df['Churn']
y.value_counts()

0    2850
1     483
Name: Churn, dtype: int64

In [4]:
#언더샘플링
from imblearn.under_sampling import RandomUnderSampler
X_sample, y_sample = RandomUnderSampler(random_state=0).fit_resample(X, y)
X_samp = pd.DataFrame(data=X_sample,columns=cols )
y_samp = pd.DataFrame(data=y_sample,columns=['Churn'])
df_samp=pd.concat([X_samp,y_samp],axis=1)
print(df_samp['Churn'].value_counts())
X=df_samp[cols]
y=df_samp['Churn']

0    483
1    483
Name: Churn, dtype: int64


In [5]:
#시간이 오래 걸림
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
# 선형모형만 가능함(rbf 커널에는 coef가 제공되지 않음)
svc = SVC(kernel="linear")
# step=1 각 단계에서 제거할 변수의 개수
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(5), scoring='accuracy')
rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=SVC(kernel='linear'), scoring='accuracy')

In [6]:
import numpy as np
print(np.mean(rfecv.grid_scores_,axis=1)) #교차검증으로 계산한 변수들의 중요도
print(rfecv.n_features_) #교차검증을 통해 선택된 변수 개수
print(rfecv.support_) #변수 선택 여부
print(rfecv.ranking_) #변수 선택 랭킹
#선택된 변수들
print(X.columns[rfecv.support_])
#선택되지 않은 변수들
print(X.columns.difference(X.columns[rfecv.support_]))
df2=pd.DataFrame(list(zip(X.columns,np.mean(rfecv.grid_scores_,axis=1))),columns=['변수명','평균점수'])
df2

[0.6097217  0.66975055 0.69358474 0.69463704 0.70392607 0.73391913
 0.74842156 0.7484269  0.75154105 0.76812136 0.76500721 0.75983121
 0.7660328  0.76603814 0.76500721 0.76500721]
10
[ True  True  True  True False False False False  True False False  True
  True  True  True  True]
[1 1 1 1 5 2 4 6 1 3 7 1 1 1 1 1]
Index(['Int'l Plan', 'VMail Plan', 'VMail Message', 'Day Mins', 'Eve Charge',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls'],
      dtype='object')
Index(['Day Calls', 'Day Charge', 'Eve Calls', 'Eve Mins', 'Night Calls',
       'Night Mins'],
      dtype='object')


c:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: The `grid_scores_` attribute is deprecated in version 1.0 in favor of `cv_results_` and will be removed in version 1.2.
  warnings.warn(msg, category=FutureWarning)


,변수명,평균점수
0,Int'l Plan,0.609722
1,VMail Plan,0.669751
2,VMail Message,0.693585
3,Day Mins,0.694637
4,Day Calls,0.703926
5,Day Charge,0.733919
6,Eve Mins,0.748422
7,Eve Calls,0.748427
8,Eve Charge,0.751541
9,Night Mins,0.768121


In [7]:
df2.sort_values(by='평균점수', ascending=False)

,변수명,평균점수
9,Night Mins,0.768121
13,Intl Calls,0.766038
12,Intl Mins,0.766033
10,Night Calls,0.765007
14,Intl Charge,0.765007
15,CustServ Calls,0.765007
11,Night Charge,0.759831
8,Eve Charge,0.751541
7,Eve Calls,0.748427
6,Eve Mins,0.748422
